## Import Modules

In [ ]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
from ble_rx_stream import *
import time
import numpy as np
import matplotlib.pyplot as plt

LOG.propagate = False

## Connect to Artemis

In [ ]:
ble = get_ble_controller()
ble.connect()

## Run Stunt

In [ ]:
time.sleep(5)
timeout_s = 10
target_m = 0.5
power_l = 255
power_r = power_l
ble.send_command(CMD.RUN_STUNT, f"{int(timeout_s*1000)}|{int(target_m*1000)}|{power_l}|{power_r}")
time.sleep(timeout_s)

In [ ]:
fut = ble_rx_stream(ble, 'RX_STREAM')
ble.send_command(CMD.DATA_STUNT, "")
stream_data = list(unpack_stream('LHh*', await fut))

In [ ]:
data = np.array(stream_data)
data_time = data[:, 0] / 1000
data_dist = data[:, 1]
data_motr = data[:, 2]

In [ ]:
_, axs = plt.subplots(2, 1, sharex=True)
axs[0].plot(data_time, data_dist)
axs[0].axhline(target_m*1000, c='red', ls='--')
axs[0].set_ylabel('distance (mm)')
axs[1].plot(data_time, data_motr)
axs[1].set_ylabel('motor value (PWM)')
plt.xlabel('time (s)')
plt.savefig('run.jpg')

## Disconnect from Artemis

In [ ]:
ble.disconnect()